*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
#except Exception:
#  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,TextVectorization
from tensorflow.keras.layers import Dense,GlobalAveragePooling1D

#print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:

val_df = pd.read_csv("valid-data.tsv",sep="\t")
train_df = pd.read_csv("train-data.tsv",sep="\t")

val_df.columns = ["state","text"]
train_df.columns = ["state","text"]

val_df = val_df.replace({"ham":1,"spam":0})
train_df = train_df.replace({"ham":1,"spam":0})


In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


In [ ]:
vectorize_layer= TextVectorization(standardize=custom_standardization,
                    max_tokens=10000,output_mode="int",output_sequence_length=1000)


In [ ]:
X_train = tf.convert_to_tensor(train_df.text)
y_train = tf.convert_to_tensor(train_df.state)

X_test = tf.convert_to_tensor(val_df.text)
y_test = tf.convert_to_tensor(val_df.state)


In [ ]:
vectorize_layer.adapt(X_train)
#vectorize_layer.adapt(X_test)

print(np.array(vectorize_layer.get_vocabulary()[:100]))


In [ ]:
model = Sequential([
    tf.keras.Input(shape=(1,),dtype=tf.string),
    vectorize_layer,
    Embedding(input_dim=len(vectorize_layer.get_vocabulary()),
              output_dim=16,mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    #GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
    
])


In [ ]:
print(model.summary())
model.compile(loss= tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=["accuracy"])

model.fit(X_train,y_train,epochs=10,batch_size=256)


In [ ]:
preds = model.predict(X_test)
preds = tf.nn.sigmoid(preds)

In [ ]:
preds,y_test

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    dic = {1:"ham",0:"spam"}
    lst = pred_text
    if not str(type(pred_text)) == "<class 'list'>":
        lst = [pred_text]
    prediction = model.predict(lst)
    prediction = tf.nn.sigmoid(prediction).numpy()
    #prediction = [[pred[0], dic[round(pred[0])]] for pred in prediction]
    return prediction[0][0], dic[round(prediction[0][0])]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]
[predict_message(mes) for mes in test_messages]

In [ ]:
str(type([]))

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
